In [4]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model
import cv2, os
from os import listdir
from os.path import isfile, join

import preprocess

IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 66, 200, 3
INPUT_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)

#from ironcar import Ironcar

In [14]:
#ironcar = Ironcar()
root = "../AutoCar/records_rev/"

files = [f for f in listdir(root) if isfile(join(root, f))]

for f in files:
    try:
        file_path = root + f
        img = cv2.imread(file_path)
        img = preprocess.preprocess(img)
        cv2.imwrite("./normal/" + f, img)

        img, blur = preprocess.blur(img)
        cv2.imwrite("./blur/" + f, blur)
    except ValueError:
        print(root + f)


/usr/local/lib/python3.5/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python3.5/dist-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


In [15]:
model_name = "./model-test.hdf5"
model = load_model(model_name)

roots = ["./normal/", "./blur/"]
res = []

for root, i in zip(roots, range(len(roots))):
    files = [root + f for f in listdir(root) if isfile(join(root, f))]
    imgs = [ img for img in map(cv2.imread, files) ]

    prediction = model.predict(np.array(imgs))
    res.append(prediction)
    
diff = np.mean(abs(res[0] - res[1]))
print(diff)

"""
for f in files:
    file_path = root + f
    img = cv2.imread(file_path)
    img = np.array([img])
    prediction.append(model.predict(img))


root = "./blur/"
files = [f for f in listdir(root) if isfile(join(root, f))]
blur_pred = []

for f in files:
    file_path = root + f
    img = mpimg.imread(file_path)
    img = np.array([img])
    blur_pred.append(model.predict(img))

print(np.array(prediction))
#print(np.array(blur_pred))
"""

/home/cai/.local/lib/python3.5/site-packages/keras/engine/saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


0.09017845


'\nfor f in files:\n    file_path = root + f\n    img = cv2.imread(file_path)\n    img = np.array([img])\n    prediction.append(model.predict(img))\n\n\nroot = "./blur/"\nfiles = [f for f in listdir(root) if isfile(join(root, f))]\nblur_pred = []\n\nfor f in files:\n    file_path = root + f\n    img = mpimg.imread(file_path)\n    img = np.array([img])\n    blur_pred.append(model.predict(img))\n\nprint(np.array(prediction))\n#print(np.array(blur_pred))\n'